In [1]:
import os
import subprocess

# Run a shell, source .bashrc, then printenv
cmd = 'bash -c "source ~/.bashrc && printenv"'
result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
env_vars = {}
for line in result.stdout.splitlines():
    if '=' in line:
        key, value = line.split('=', 1)
        env_vars[key] = value

# Now, update os.environ with these
os.environ.update(env_vars)

In [2]:
import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Redirect all outputs here
hide = widgets.Output()

def display_gds(gds_file, path,scale = 3):
  
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg(os.path.join(path,'out.svg'))
    
  # Scale the image for displaying
  fig = sg.fromfile(os.path.join(path,'out.svg'))
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save(os.path.join(path,'out.svg'))

  # Display the image
  IPython.display.display(IPython.display.SVG(os.path.join(path,'out.svg')))
  os.remove(os.path.join(path,'out.gds'))

def display_component(component,path,scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds(os.path.join(path,'out.gds'))
  display_gds(os.path.join(path,'out.gds'),path,scale)

In [3]:
from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.components import text_freetype, rectangle

In [4]:
from glayout import nmos, pmos
from glayout import via_stack
from glayout import rename_ports_by_orientation
from glayout import tapring

In [5]:
from glayout.util.comp_utils import evaluate_bbox, prec_center, prec_ref_center, align_comp_to_port
from glayout.util.port_utils import add_ports_perimeter,print_ports
from glayout.util.snap_to_grid import component_snap_to_grid
from glayout.spice.netlist import Netlist

In [6]:
from glayout.routing.straight_route import straight_route
from glayout.routing.c_route import c_route
from glayout.routing.L_route import L_route

In [7]:
nmos_kwargs = {
    "with_tie": True,
    "with_dnwell": True,
    "sd_route_topmet": "met2",
    "gate_route_topmet": "met2",
    "sd_route_left": True,
    "rmult": None,
    "gate_rmult": 1,
    "interfinger_rmult": 1,
    "substrate_tap_layers": ("met2","met1"),
    "dummy_routes": True
}

pmos_kwargs = {
    "with_tie": True,
    "dnwell": False,
    "sd_route_topmet": "met2",
    "gate_route_topmet": "met2",
    "sd_route_left": True,
    "rmult": None,
    "gate_rmult": 1,
    "interfinger_rmult": 1,
    "substrate_tap_layers": ("met2","met1"),
    "dummy_routes": True
}

In [8]:
def currentMirrorPmosref(pdk, W=1, L=1, fing = 1, mult = 1):
  currMirrComp = Component()
  pfet_ref = pmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap=False, with_dummy=(False, True), with_tie = False)
  
  cmirr_ref = currMirrComp << pfet_ref
  cmirr_ref.mirror_x()

    
    
  pdk.util_max_metal_seperation()

  spacing = pdk.util_max_metal_seperation() 

  tap_ring = tapring(pdk, enclosed_rectangle=evaluate_bbox(currMirrComp.flatten(), padding=pdk.get_grule("nwell", "active_diff")["min_enclosure"]))
  shift_amount = -prec_center(currMirrComp.flatten())[0]
  tring_ref = currMirrComp << tap_ring
  tring_ref.movex(destination=shift_amount)
  #Conexion de referencia  
  currMirrComp << c_route(pdk, cmirr_ref.ports["multiplier_0_gate_E"], cmirr_ref.ports["multiplier_0_drain_E"])

  return currMirrComp

#curr_mirr_P = currentMirrorPmos(gf180).write_gds("cmirror_example.gds")
#display_gds("cmirror_example.gds")
curr_ref_P= currentMirrorPmosref(gf180, W=0.5, L=0.28, fing = 1, mult = 1)
curr_ref_P.show()

2025-09-12 09:21:15.193 | INFO     | gdsfactory.pdk:activate:337 - 'gf180' PDK is now active
/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 3 in 'Unnamed_d987cb1d'
  gdspath = component.write_gds(
2025-09-12 09:21:17.763 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.4.1", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_d987cb1d.gds"}


In [9]:
def currentMirrorPmos_noref(pdk, W=1, L=1, fing = 1, mult = 1):
  currMirrComp = Component()
  pfet_mir = pmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap=False, with_dummy=(False, True), with_tie = False)
  pfet_mir_2 = pmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap = False, with_dummy=(False, False), with_tie = False)
  pfet_mir_3 = pmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap = False, with_dummy=(False, False), with_tie = False)
  pfet_mir_4 = pmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap = False, with_dummy=(False, True), with_tie = False)
 
 
  cmir_ref = currMirrComp << pfet_mir
  cmir_ref_2 = currMirrComp << pfet_mir_2
  cmir_ref_3 = currMirrComp << pfet_mir_3
  cmir_ref_4 = currMirrComp << pfet_mir_4

  cmir_ref.mirror_x()
  cmir_ref_2.mirror_x()
    
    
  pdk.util_max_metal_seperation()

  spacing = pdk.util_max_metal_seperation()
  k = 4.13 #Este valor tiene que ver con el dummy extra que se agrega al parecer  
  cmir_ref_2.movex(evaluate_bbox(pfet_mir)[0] - k*spacing) 
  cmir_ref_3.movex(evaluate_bbox(pfet_mir)[0] - k*spacing  + evaluate_bbox(pfet_mir_2)[0])  
  cmir_ref_4.movex(evaluate_bbox(pfet_mir)[0] - k*spacing   + evaluate_bbox(pfet_mir_2)[0] + evaluate_bbox(pfet_mir_3)[0] + k*spacing)



  tap_ring = tapring(pdk, enclosed_rectangle=evaluate_bbox(currMirrComp.flatten(), padding=pdk.get_grule("nwell", "active_diff")["min_enclosure"]))
  shift_amount = -prec_center(currMirrComp.flatten())[0]
  tring_ref = currMirrComp << tap_ring
  tring_ref.movex(destination=shift_amount)
  #Conexiones Source2-Copy1.ipynb
  currMirrComp << straight_route(pdk, cmir_ref.ports["multiplier_0_source_E"], cmir_ref_2.ports["multiplier_0_source_E"])
  currMirrComp << straight_route(pdk, cmir_ref_2.ports["multiplier_0_source_E"], cmir_ref_3.ports["multiplier_0_source_E"])
  currMirrComp << straight_route(pdk, cmir_ref_3.ports["multiplier_0_source_E"], cmir_ref_4.ports["multiplier_0_source_E"])
  #Conexiones Gate
  currMirrComp << straight_route(pdk, cmir_ref.ports["multiplier_0_gate_E"], cmir_ref_2.ports["multiplier_0_gate_E"])
  currMirrComp << straight_route(pdk, cmir_ref_2.ports["multiplier_0_gate_E"], cmir_ref_3.ports["multiplier_0_gate_E"])
  currMirrComp << straight_route(pdk, cmir_ref_3.ports["multiplier_0_gate_E"], cmir_ref_4.ports["multiplier_0_gate_E"])

  return currMirrComp

#curr_mirr_P = currentMirrorPmos(gf180).write_gds("cmirror_example.gds")
#display_gds("cmirror_example.gds")
curr_mirr_P = currentMirrorPmos_noref(gf180, W=1, L=0.28, fing = 10, mult = 4)
curr_mirr_P.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 6 in 'Unnamed_0456d3d5'
  gdspath = component.write_gds(
2025-09-12 09:21:38.825 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.4.1", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_0456d3d5.gds"}


In [10]:
def currentMirrorNmos(pdk, W=1, L=1, fing = 1, mult = 1):
  currMirrComp = Component()
    
  nfet_ref = nmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap=False, with_dummy=(False, True), with_tie = False)
  nfet_mir = nmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  nfet_mir_2 = nmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap = False, with_dummy=(False, False), with_tie = False)
  nfet_mir_3 = nmos(pdk, width = W, length = L, multipliers = mult, fingers = fing, with_substrate_tap = False, with_dummy=(False, True), with_tie = False)
 
  cref_ref = currMirrComp << nfet_ref
  cmir_ref = currMirrComp << nfet_mir
  cmir_ref_2 = currMirrComp << nfet_mir_2
  cmir_ref_3 = currMirrComp << nfet_mir_3

  cref_ref.mirror_x()
  cmir_ref.mirror_x()
    
    
  pdk.util_max_metal_seperation()

  spacing = pdk.util_max_metal_seperation()
  k = 4.335 #Esto tiene que ver con la distancia total incluyendo un dummy
  cmir_ref.movex(evaluate_bbox(nfet_ref)[0] - k*spacing)  
  cmir_ref_2.movex(evaluate_bbox(nfet_ref)[0] - k*spacing + evaluate_bbox(nfet_mir)[0])  
  cmir_ref_3.movex(evaluate_bbox(nfet_ref)[0] - k*spacing   + evaluate_bbox(nfet_mir)[0] + evaluate_bbox(nfet_mir_2)[0] + k*spacing)
    

  tap_ring = tapring(pdk, enclosed_rectangle=evaluate_bbox(currMirrComp.flatten(), padding=pdk.get_grule("pwell", "active_diff")["min_enclosure"]))
  shift_amount = -prec_center(currMirrComp.flatten())[0]
  tring_ref = currMirrComp << tap_ring
  tring_ref.movex(destination=shift_amount)
  #Conexiones Source
  currMirrComp << straight_route(pdk, cref_ref.ports["multiplier_0_drain_W"], cmir_ref.ports["multiplier_0_drain_W"])
  currMirrComp << straight_route(pdk, cmir_ref.ports["multiplier_0_drain_W"], cmir_ref_2.ports["multiplier_0_drain_W"])
  currMirrComp << straight_route(pdk, cmir_ref_2.ports["multiplier_0_drain_W"], cmir_ref_3.ports["multiplier_0_drain_W"])
  #Conexiones Gate
  currMirrComp << straight_route(pdk, cref_ref.ports["multiplier_0_gate_E"], cmir_ref.ports["multiplier_0_gate_E"])
  currMirrComp << straight_route(pdk, cmir_ref.ports["multiplier_0_gate_E"], cmir_ref_2.ports["multiplier_0_gate_E"])
  currMirrComp << straight_route(pdk, cmir_ref_2.ports["multiplier_0_gate_E"], cmir_ref_3.ports["multiplier_0_gate_E"])

  return currMirrComp

#curr_mirr_N = currentMirrorNmos(gf180).write_gds("cmirror_example.gds")
#display_gds("cmirror_example.gds")
curr_mirr_N = currentMirrorNmos(gf180, W = 1, L = 0.28, mult = 4, fing = 10)
curr_mirr_N = curr_mirr_N.rotate(180)
curr_mirr_N.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 6 in 'rotate_165b74f5'
  gdspath = component.write_gds(
2025-09-12 09:21:59.613 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.4.1", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/rotate_165b74f5.gds"}


In [11]:
def inv_vco(pdk, Wp=1, Lp=1, Wn=1, Ln=1, mult=1, fingp=1, fingn=1):

  invTop = Component()
  
  invComp = Component()
  pfet_i0 = pmos(pdk, width = Wp, length = Lp, multipliers = mult, fingers = fingp, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  pfet_i1 = pmos(pdk, width = Wp, length = Lp, multipliers = mult, fingers = fingp, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  pfet_i2 = pmos(pdk, width = Wp, length = Lp, multipliers = mult, fingers = fingp, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  
  p_i0 = invComp << pfet_i0
  p_i1 = invComp << pfet_i1
  p_i2 = invComp << pfet_i2

  #p_i0.mirror_x()

  pdk.util_max_metal_seperation()

  #PMOS Placement    
  spacing = pdk.util_max_metal_seperation()
  k = 0 #Este valor tiene que ver con el dummy extra que se agrega al parecer  
  p_i1.movex(evaluate_bbox(p_i0)[0] - k*spacing) 
  p_i2.movex(evaluate_bbox(p_i0)[0] - k*spacing  + evaluate_bbox(p_i1)[0] + k*spacing)

  #PMOS Tap Ring
  tap_ringp = tapring(pdk, enclosed_rectangle=evaluate_bbox(invComp.flatten(), padding=pdk.get_grule("nwell", "active_diff")["min_enclosure"]))
  shift_amount = -prec_center(invComp.flatten())[0]
  tringp_ref = invComp << tap_ringp
  tringp_ref.movex(destination=shift_amount)


  invCompN = Component()
  pfet_i0 = pmos(pdk, width = Wp, length = Lp, multipliers = mult, fingers = fingp, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  nfet_i0 = nmos(pdk, width = Wn, length = Ln, multipliers = mult, fingers = fingn, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  nfet_i1 = nmos(pdk, width = Wn, length = Ln, multipliers = mult, fingers = fingn, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  nfet_i2 = nmos(pdk, width = Wn, length = Ln, multipliers = mult, fingers = fingn, with_substrate_tap=False, with_dummy=(False, False), with_tie = False)
  
  n_i0 = invCompN << nfet_i0
  n_i1 = invCompN << nfet_i1
  n_i2 = invCompN << nfet_i2

  n_i0.mirror_y()
  n_i1.mirror_y()
  n_i2.mirror_y()
    
  pdk.util_max_metal_seperation()
  ref_dimensions = evaluate_bbox(nfet_i0)

  y_offset_NMOS = 2
    
  #Placing NMOS downwards
  n_i0.movey(pfet_i0.ymin - ref_dimensions[1]/2 - pdk.util_max_metal_seperation()- y_offset_NMOS)
  n_i1.movey(pfet_i0.ymin - ref_dimensions[1]/2 - pdk.util_max_metal_seperation()- y_offset_NMOS)
  n_i2.movey(pfet_i0.ymin - ref_dimensions[1]/2 - pdk.util_max_metal_seperation()- y_offset_NMOS)

  #NMOS Placement    
  x_offset_NMOS = 5 #-2.5 lo manda al extremo izquierdo del pmos
  spacing = pdk.util_max_metal_seperation()
  k1 = 0 #Este valor tiene que ver con el dummy extra que se agrega al parecer 
  n_i0.movex(-x_offset_NMOS)
  n_i1.movex(evaluate_bbox(n_i0)[0] - k1*spacing - x_offset_NMOS) 
  n_i2.movex(evaluate_bbox(n_i0)[0] - k1*spacing  + evaluate_bbox(n_i1)[0] + k1*spacing - x_offset_NMOS) 
  
  
    
  #NMOS Tap Ring
  tap_ringn = tapring(pdk, enclosed_rectangle=evaluate_bbox(invCompN.flatten(), padding=pdk.get_grule("pwell", "active_diff")["min_enclosure"]))
  shift_amount = -prec_center(invCompN.flatten())[0]
  tringn_ref = invCompN << tap_ringn
  tringn_ref.movex(destination=shift_amount)
  tringn_ref.movey(pfet_i0.ymin - ref_dimensions[1]/2 - pdk.util_max_metal_seperation()- y_offset_NMOS)  



  #VIAS
  viam2m3 = via_stack(pdk, "met2", "met3", centered=True) #met2 is the bottom layer. met3 is the top layer.
  #we need four such vias
  #NMOS0
  dn1 = invCompN << viam2m3
  gn1 = invCompN << viam2m3
  #NMOS1
  dn2_1 = invCompN << viam2m3
  dn2_2 = invCompN << viam2m3
  #PMOS1
  dp2_1 = invCompN << viam2m3
  dp2_2 = invCompN << viam2m3
  dn2_3 = invCompN << viam2m3
  #NMOS3
  gn3 = invCompN << viam2m3   
  
  dn1.move(n_i1.ports["multiplier_0_drain_W"].center).movey(4.14)
  dn1.movex(-8.23)
  gn1.move(n_i1.ports["multiplier_0_gate_N"].center).movey(1.5)
  dn2_1.move(n_i1.ports["multiplier_0_drain_W"].center)
  dn2_2.move(n_i1.ports["multiplier_0_drain_W"].center).movey(-1.8)
  dp2_1.move(p_i1.ports["multiplier_0_drain_W"].center)
  dp2_2.move(p_i1.ports["multiplier_0_drain_W"].center).movey(2)
  dn2_3.move(n_i1.ports["multiplier_0_drain_W"].center).movex(-9.9) 
  dn2_3.movey(-1.8)
  gn3.move(p_i2.ports["multiplier_0_gate_E"].center).movex(4.75)

  #Routing
    
  #Falta Conexion entre etapas, drain del primero con gate del segundo
  #invTop << c_route(pdk, p_i0.ports["multiplier_0_gate_W"], n_i0.ports["multiplier_0_drain_W"])
  invTop << straight_route(pdk, dn1.ports["top_met_W"], gn1.ports["top_met_E"])
  invTop << straight_route(pdk, dn2_1.ports["top_met_W"], dn2_2.ports["top_met_E"])
  invTop << straight_route(pdk, dn2_1.ports["bottom_met_S"], dn2_2.ports["bottom_met_N"])
  invTop << L_route(pdk, gn3.ports["top_met_N"], dp2_2.ports["bottom_met_W"])
    
  #Conexion entre source de todos los PMOS
  invTop << straight_route(pdk, p_i0.ports["multiplier_0_source_E"], p_i1.ports["multiplier_0_source_E"])
  invTop << straight_route(pdk, p_i1.ports["multiplier_0_source_E"], p_i2.ports["multiplier_0_source_E"])

  #Conexion entre source de todos los NMOS
  invTop << straight_route(pdk, n_i0.ports["multiplier_0_source_E"], n_i1.ports["multiplier_0_source_E"])
  invTop << straight_route(pdk, n_i1.ports["multiplier_0_source_E"], n_i2.ports["multiplier_0_source_E"])

  invCompN = invCompN.rotate(0)
  invTop << invComp
  invTop << invCompN

  #Conexion entre drains de NMOS con PMOS
  invTop << c_route(pdk, p_i0.ports["multiplier_0_drain_W"], n_i0.ports["multiplier_0_drain_W"])
  #invTop << c_route(pdk, p_i1.ports["multiplier_0_drain_N"], n_i1.ports["multiplier_0_drain_S"])
  invTop << c_route(pdk, p_i2.ports["multiplier_0_drain_E"], n_i2.ports["multiplier_0_drain_E"])
  invTop << straight_route(pdk, dp2_1.ports["top_met_N"], dp2_2.ports["top_met_S"])
  invTop << straight_route(pdk, dn2_2.ports["bottom_met_W"], dn2_3.ports["bottom_met_W"])
  invTop << L_route(pdk, dn2_3.ports["top_met_N"], dp2_2.ports["bottom_met_W"])
  #Conexion entre gates de NMOS con PMOS
  invTop << L_route(pdk, p_i0.ports["multiplier_0_gate_W"], n_i0.ports["multiplier_0_gate_S"])
  invTop << straight_route(pdk, p_i1.ports["multiplier_0_gate_N"], n_i1.ports["multiplier_0_gate_S"])
  invTop << L_route(pdk, p_i2.ports["multiplier_0_gate_W"], n_i2.ports["multiplier_0_gate_S"])

  return invTop

inv_vco = inv_vco(gf180, Wp=0.5, Lp=0.28, Wn=0.5, Ln=0.28, mult=1, fingp=1, fingn=1)
inv_vco.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 11 in 'Unnamed_7b574d1a'
  gdspath = component.write_gds(
2025-09-12 09:22:07.868 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.4.1", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_7b574d1a.gds"}


In [12]:
def main(inv_vco,currentMirrorNMOS, currentMirrorPmos_noref, currentMirrorPmosref):
    top_vco = Component()
    top_vco << inv_vco
    top_vco << currentMirrorNMOS.movex(-10)
    top_vco << currentMirrorPmos_noref.movey(-10)
    top_vco << currentMirrorPmosref.movex(10)

    return top_vco

top_vco = main(inv_vco,curr_mirr_N, curr_mirr_P, curr_ref_P)
top_vco.show()


ValueError: 
You cannot move a Component. You can create a new Component, add a reference to the other Component and then move the reference.

For example:

# BAD
c = gf.components.straight()
c.xmin = 10

# GOOD
c = gf.Component()
ref = c.add_ref(gf.components.straight()) # or ref = c << gf.components.straight()
ref.xmin = 10
